In [ ]:
%pip install contextily geopandas openpyxl

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt
from pathlib import Path

import os
import sys
sys.path.append(os.path.join(os.path.abspath(".."), "functions"))

import data_reader, spatial_assignment
from constants import P2V


In [ ]:
# Set up constants
data_dir = Path('.').absolute() / 'data'

In [ ]:
# Load Area Verde (AV) shape
green_area = data_reader.AV_shape(namefile="area_verde_manual_v1.geojson", datapath=data_dir)

# Load OD shapes and clean overlapping zones
aree_gdf = data_reader.OD_shapes(namefile_polygons="Shape_zone.SHP", namefile_centers="Shape_zone_centroid.SHP", datapath=data_dir)

In [ ]:
# Link OD to AV: mostly-overlapping zones
aree_gdf = spatial_assignment.OD_to_AV(df_od=aree_gdf, df_av=green_area)

#Link OD to AV: intersecting zones
aree_any_intersects = aree_gdf.sjoin(
    green_area, predicate="intersects", how="inner"
)
aree_gdf[["intersects_area_verde"]] = False
aree_gdf.loc[aree_any_intersects.index, "intersects_area_verde"] = True


In [ ]:
fig, ax = plt.subplots()

aree_gdf.plot(ax=ax,column='mostly_within_area_verde')
green_area.boundary.plot(ax=ax, color='black', alpha=0.5)

ctx.add_basemap(ax, crs=green_area.crs, source=ctx.providers.OpenStreetMap.Mapnik, alpha=0.7)


In [ ]:
# Load OD flows -> only those with a shape, either polygonal or point
# (unit: people)
od = data_reader.OD_flows(namefile="PROGETTO-OD.xlsx", datapath=data_dir)

print(f"total people: {od['flow'].sum()}")
print(f"total vehicles: {od['flow'].sum() * P2V} --- computed using rescaling P2V={round(P2V,4)}")

In [ ]:
# Aggregate flows based on Area Verde intersections
od = (
    od
    .merge(aree_gdf[['id', 'mostly_within_area_verde']].rename(columns={'id': 'from', 'mostly_within_area_verde': 'from_area_verde'}), on='from', how='left')
    .merge(aree_gdf[['id', 'mostly_within_area_verde']].rename(columns={'id': 'to', 'mostly_within_area_verde': 'to_area_verde'}), on='to', how='left')
)

In [ ]:
#Total flows (unit: people)
df_check = od.groupby(['from_area_verde', 'to_area_verde'])['flow'].sum().reset_index()
df_check


In [ ]:
# Comparison with previous results, with different shapefiles of input
# (unit: people)
df_check['flow_before'] = [1.0326e6, 269598.56,268077.238, 322006.47]
df_check